# Team

<a target="_blank" href="https://colab.research.google.com/github/glaucogoncalves/nio/blob/main/assignments/lp-exec-2.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Helder Mateus dos Reis Matos

# Attention, please!

Solve the problems below. Show the problem model and the solution using ```pyomo``` and the GLPK or COIN-OR solvers.

To see how to write optimization models in colab, look for examples in the Linear Programming class colab.

Please send your Colab file (.ipynb) in SIGAA.

In [1]:
# !pip install -q pyomo
# !apt install -q -y glpk-utils

In [2]:
# imports

import pyomo.environ as pyo

# Problem 1

A furniture manufacturer employs 6 skilled and 11 semiskilled workers and produces two products: study table and computer table. A study table requires 2 h of a skilled worker and 2 h of an unskilled worker. A computer table requires 2 h of a skilled worker and 5 h of an unskilled worker. As per the industrial laws, no one is allowed to work for more than 38 h a week. The manufacturer can sell as many tables as he can produce. If the profit for a study table is $100 and for a computer table $160, how many study and computer tables should the manufacturer produce in a week in order to maximize the overall profit? Formulate a linear programming model and solve it.

## 1.1. Objective function

The objective function for problem 1 should maximize the profit of the products. The decision variables $st$ and $ct$ store the number of study tables and computer tables produced, respectively, which constitute the following objective function:

$$\text{max } 100 st + 160 ct$$

## 1.2. Constraints

The main constraint given by the problem is related to the maximum weekly hours for a worker, which is 38h. As the objective function is taken as a function of the number of each type of table, the maximum working hours should also be taken as a function of the number of tables. But before writing the constraint, it is important to generalize the number of hours that each type of employer should work:

- Skilled workers: $6 \times 38 = 228$ hours
- Unskilled workers: $11 \times 38 = 418$ hours

The resulting hours are the restriction for whatever number of workers that would be assigned. Other important aspect is that there is no such thing as "negative tables" nor "negative hours", so the number of tables are always positive. As such, these are the constraints.

$$\text{subject to }\left\{
    \begin{array}{ll}
        2st + 2ct \leq 228, \\
        2st + 5ct \leq 418, \\
        st \geq 0, \\
        ct \geq 0.
    \end{array}
\right.$$

## 1.3. PyOMO model

I'll define variables for the profit of each type of table and the maximum amount of hours for each type of worker.

In [3]:
profit_st = 100
profit_ct = 160

skilled_hours = 6 * 38
semiskilled_hours = 11 * 38

Then a `ConcreteModel` is created to store information regarding the problem, where the decision variables $st$ and $ct$ are assigned as positive floating numbers.

In [4]:
model_manufacturer = pyo.ConcreteModel()
model_manufacturer.st = pyo.Var(domain=pyo.NonNegativeReals)
model_manufacturer.ct = pyo.Var(domain=pyo.NonNegativeReals)

The objective function is stored as an attribute called `profit`, designed to be maximized.

In [5]:
model_manufacturer.profit = pyo.Objective(expr=profit_st * model_manufacturer.st + profit_ct * model_manufacturer.ct, sense=pyo.maximize)

The time constraints are added, following their algebraic expression.

In [6]:
model_manufacturer.skill = pyo.Constraint(expr=2 * model_manufacturer.st + 2 * model_manufacturer.ct <= skilled_hours)
model_manufacturer.semi  = pyo.Constraint(expr=2 * model_manufacturer.st + 5 * model_manufacturer.ct <= semiskilled_hours)

That concludes the modeling with PyOMO. The next step is solving the linear programming problem.

## 1.4. Solving with GLPK

I chose GLPK as the solver for its integration with PyOMO's SolverFactory, which only needs the specification of the name of the solver. From now on, I'll use the object `solver` for the other exercises. Information of the solution can be obtained via the display method of `model_manufacturer`.

The maximum profit obtained is $15199.99$, produced by $st = 50.6$ study tables and $ct = 63.3$ computer tables. As for the constraits, all of the available 228 hours for skilled and 418 hours for unskilled workers were used, despite the floating point errors produced by the tool. Considering that no fractional tables should be crafted, 51 study tables and 63 computer tables would generate a profit of $15180$.

In [7]:
solver = pyo.SolverFactory("glpk")
solver.solve(model_manufacturer, tee=False)

model_manufacturer.display()

Model unknown

  Variables:
    st : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 50.6666666666667 :  None : False : False : NonNegativeReals
    ct : Size=1, Index=None
        Key  : Lower : Value            : Upper : Fixed : Stale : Domain
        None :     0 : 63.3333333333333 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 15199.999999999998

  Constraints:
    skill : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 228.0 : 228.0
    semi : Size=1
        Key  : Lower : Body              : Upper
        None :  None : 417.9999999999999 : 418.0


# Problem 2

CPS Investment Services must develop an investment portfolio for a new customer. Initially, the new customer would like to restrict portfolio to three stocks as shown below:

|Stock|Price per Share ($)|Estimated Annual Return (%)|
|--|--|--|
|ACT cable|50|6|
|CTR retail|35|10|
|PCL petroleum|120|8|

The customer wants to invest $80,000 and established two investment
goals.
Goal 1: Obtain an annual return of at least 9%.
Goal 2: Limit the investment in CTR retail, the riskier investment, to no more than 50% of the total investment.

Formulate a mathematical programming model for the CPS investment problem and solve it.

## 2.1. Objective function

The objective function for problem 2 should maximize annual return. The decision variables $ACT$, $CTR$ and $PCL$ store the amount of money invested in cable, retail and petroleum, respectively, which along their proportional estimated return constitute the following objective function:

$$\text{max } 0.06 \times ACT + 0.1 \times CTR + 0.08 \times PCL$$

## 2.2. Constraints

As a consequence of the objective function, total investment should be alocated to each customer:

$$ACT + CTR + PCL \leq 80000$$

To achive goal 1, you should notice that the return must be at least 9% of the investment of $80,000:

$$0.09 \times 80000 = 7200$$

To achieve goal 2, you should notice that the investiment in CTR should be lower than half of the total investment:

$$CTR \leq 0.5 \times 80000$$
$$CTR \leq 40000$$

Also, there is no such thing as a negative investment. As such, these are the constraints.

$$\text{subject to }\left\{
    \begin{array}{ll}
        ACT + CTR + PCL \leq 80000, \\
        0.09 \times 80000 = 7200, \\
        CTR \leq 40000, \\
        ACT \geq 0, \\
        CTR \geq 0, \\
        PCL \geq 0, \\
    \end{array}
\right.$$

## 2.3. PyOMO model

I'll define variables for the total investment and the estimated return.

In [8]:
total = 80000

act = 0.06
ctr = 0.10
pcl = 0.08

Then a `ConcreteModel` is created to store information regarding the problem, where the decision variables $act$, $ctr$ and $pcl$ are assigned as positive floating numbers.

In [9]:
model_investment = pyo.ConcreteModel()
model_investment.act = pyo.Var(domain=pyo.NonNegativeReals)
model_investment.ctr = pyo.Var(domain=pyo.NonNegativeReals)
model_investment.pcl = pyo.Var(domain=pyo.NonNegativeReals)

The objective function is stored as an attribute called `profit`, designed to be maximized.

In [10]:
model_investment.annual_return = pyo.Objective(expr=act*model_investment.act + ctr*model_investment.ctr + pcl*model_investment.pcl, sense=pyo.maximize)

The time constraints are added, following their algebraic expression.

In [11]:
model_investment.budget = pyo.Constraint(expr=model_investment.act + model_investment.ctr + model_investment.pcl == total)
model_investment.ctr_limit = pyo.Constraint(expr=model_investment.ctr <= 0.5*total)
model_investment.min_return = pyo.Constraint(expr=act*model_investment.act + ctr*model_investment.ctr + pcl*model_investment.pcl >= 0.09*total)

That concludes the modeling with PyOMO. The next step is solving the linear programming problem.

## 2.4. Solving with GLPK

The maximum annual return obtained is $7200$, produced by $act = 0$, $ctr = 40000$ and $pcl = 40000$. As $ctr$ is the stock with the highest return, it was filled up to its limit, then $pcl$ was filled, and then no investment remained for $act$. As for the constraints, all the budget was used, $ctr$ was invested up to its limit, and the 9% expected over the total investment was fulfilled.

In [12]:
solver = pyo.SolverFactory("glpk")
solver.solve(model_investment, tee=False)

model_investment.display()

Model unknown

  Variables:
    act : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    ctr : Size=1, Index=None
        Key  : Lower : Value   : Upper : Fixed : Stale : Domain
        None :     0 : 40000.0 :  None : False : False : NonNegativeReals
    pcl : Size=1, Index=None
        Key  : Lower : Value   : Upper : Fixed : Stale : Domain
        None :     0 : 40000.0 :  None : False : False : NonNegativeReals

  Objectives:
    annual_return : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 7200.0

  Constraints:
    budget : Size=1
        Key  : Lower   : Body    : Upper
        None : 80000.0 : 80000.0 : 80000.0
    ctr_limit : Size=1
        Key  : Lower : Body    : Upper
        None :  None : 40000.0 : 40000.0
    min_return : Size=1
        Key  : Lower  : Body   : Upper
        None : 7200.0 : 7200.0 :  None


# Problem 3

After a major bushfire, the local government is planning to establish a new fire station to cover four major town centers, under frequent fire threat, located in Town-1 (10, 20), Town-5 (50, 20), Town-8 (40, 40), and Town-14 (70, 60). Town-1 averages 20 fires per year; Town-5, 30 fires; Town-8, 40 fires; and Town-14, 25 fires. The local government wants to build the fire station in a location that minimizes the average distance that a fire engine must travel to respond to a fire. Develop a mathematical model to determine the location of the fire station.

## 3.1. Objective function

The objective function for problem 3 should minimize the average distance between a fire engine to a major town center. Consider that $(x,y)$ are the position of the new fire station, and $(x_i,y_i)$, where $i \in \{1,5,8,14\}$, are the positions of the town centers. The decision variables $u = |x-x_i|$, $v = |y-y_i|$ and $w_i$ t

$PCL$ store the amount of money invested in cable, retail and petroleum, respectively, which along their proportional estimated return constitute the following objective function:

$$\text{max } 0.06 \times ACT + 0.1 \times CTR + 0.08 \times PCL$$

## 3.2. Constraints

As a consequence of the objective function, total investment should be alocated to each customer:

$$ACT + CTR + PCL \leq 80000$$

To achive goal 1, you should notice that the return must be at least 9% of the investment of $80,000:

$$0.09 \times 80000 = 7200$$

To achieve goal 2, you should notice that the investiment in CTR should be lower than half of the total investment:

$$CTR \leq 0.5 \times 80000$$
$$CTR \leq 40000$$

Also, there is no such thing as a negative investment. As such, these are the constraints.

$$\text{subject to }\left\{
    \begin{array}{ll}
        ACT + CTR + PCL \leq 80000, \\
        0.09 \times 80000 = 7200, \\
        CTR \leq 40000, \\
        ACT \geq 0, \\
        CTR \geq 0, \\
        PCL \geq 0, \\
    \end{array}
\right.$$

## 3.3. PyOMO model

I'll define variables for the total investment and the estimated return.

In [ ]:
total = 80000

act = 0.06
ctr = 0.10
pcl = 0.08

Then a `ConcreteModel` is created to store information regarding the problem, where the decision variables $act$, $ctr$ and $pcl$ are assigned as positive floating numbers.

In [ ]:
model_investment = pyo.ConcreteModel()
model_investment.act = pyo.Var(domain=pyo.NonNegativeReals)
model_investment.ctr = pyo.Var(domain=pyo.NonNegativeReals)
model_investment.pcl = pyo.Var(domain=pyo.NonNegativeReals)

The objective function is stored as an attribute called `profit`, designed to be maximized.

In [ ]:
model_investment.annual_return = pyo.Objective(expr=act*model_investment.act + ctr*model_investment.ctr + pcl*model_investment.pcl, sense=pyo.maximize)

The time constraints are added, following their algebraic expression.

In [ ]:
model_investment.budget = pyo.Constraint(expr=model_investment.act + model_investment.ctr + model_investment.pcl == total)
model_investment.ctr_limit = pyo.Constraint(expr=model_investment.ctr <= 0.5*total)
model_investment.min_return = pyo.Constraint(expr=act*model_investment.act + ctr*model_investment.ctr + pcl*model_investment.pcl >= 0.09*total)

That concludes the modeling with PyOMO. The next step is solving the linear programming problem.

## 3.4. Solving with GLPK

The maximum annual return obtained is $7200$, produced by $act = 0$, $ctr = 40000$ and $pcl = 40000$. As $ctr$ is the stock with the highest return, it was filled up to its limit, then $pcl$ was filled, and then no investment remained for $act$. As for the constraints, all the budget was used, $ctr$ was invested up to its limit, and the 9% expected over the total investment was fulfilled.

In [ ]:
solver = pyo.SolverFactory("glpk")
result_investment = solver.solve(model_investment, tee=False)

model_investment.display()

Model unknown

  Variables:
    act : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    ctr : Size=1, Index=None
        Key  : Lower : Value   : Upper : Fixed : Stale : Domain
        None :     0 : 40000.0 :  None : False : False : NonNegativeReals
    pcl : Size=1, Index=None
        Key  : Lower : Value   : Upper : Fixed : Stale : Domain
        None :     0 : 40000.0 :  None : False : False : NonNegativeReals

  Objectives:
    annual_return : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 7200.0

  Constraints:
    budget : Size=1
        Key  : Lower   : Body    : Upper
        None : 80000.0 : 80000.0 : 80000.0
    ctr_limit : Size=1
        Key  : Lower : Body    : Upper
        None :  None : 40000.0 : 40000.0
    min_return : Size=1
        Key  : Lower  : Body   : Upper
        None : 7200.0 : 7200.0 :  None


# Problem 4

A popular product is produced at three plants (PL1, PL2, PL3) and is required to ship to three warehouses: WH1, WH2, and WH3. The supply from each plant, the delivery required in each warehouse, and the unit transportation cost from each plant to each warehouse is shown below:

|Plant|WH1|WH2|WH3|Plant Capacity|
|--|--|--|--|--|
|PL1|20|15|25|400|
|PL2|10|12|8|500|
|PL3|14|20|12|200|
|Warehouse demand|300|200|600||

Develop an LP model for minimizing the transportation cost while not violating the supply and demand constraints.

# Problem 5

A certain paint can be produced using four different production processes (P1, P2, P3, and P4). The processing cost of each liter in any of the four available processes, the maximum capacity of each process, and the setup costs are given below:

|Process|Setup Cost (\$)|Processing Cost (\$/L)|Capacity (L)|
|--|--|--|--|
|P1|5,000|0.60|20,000|
|P2|6,000|0.50|15,000|
|P3|10,000|0.40|40,000|
|P4|6,000|0.40|25,000|

Assume that a daily demand of 45,000 L must be fulfilled. Formulate the problem as a programming model to determine the daily production schedule in order to minimize total costs.